# Proyecto OCR

#
**Librerias**

In [ ]:
import fitz
from PIL import Image
import requests
from flask import Flask, request
from io import BytesIO
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
import pyodbc
print(pyodbc.drivers())
from transformers import pipeline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch

#
**Extraer textos de PDFs**

In [ ]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

pdf_text = extract_text_from_pdf(r"C:\Users\oswal\Desktop\EpicGames\IP Partner Licensing Agreement.pdf")
print(pdf_text)

#
**Extraer datos de imagenes**

**Imagen de ejemplo**

In [ ]:
image_2_show = mpimg.imread(r"C:\Users\oswal\text_image_orc.jpg")
plt.imshow(image_2_show)
plt.axis('off')
plt.show()

In [ ]:
def extract_text_from_image(image_path):
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image)
    return text

image_text = extract_text_from_image(r"C:\Users\oswal\text_image_orc.jpg")
print(f"La imagen dice: {image_text}")

#
**Conexión a SQL Server**

In [ ]:
driver_name = "ODBC Driver 17 for SQL Server"
server_ip = "192.168.1.3"
port = 1433
database_name = "documents_database"
database_user = "bricksbreaker"
database_password = "puchi15063022"

connection_str = (
    f"DRIVER={{{driver_name}}};"
    f"SERVER={server_ip},{port};"
    f"DATABASE={database_name};"
    f"UID={database_user};"
    f"PWD={database_password};"
)

connection = pyodbc.connect(connection_str)
print("Conectado correctamente") 

In [ ]:
cursor = connection.cursor()

cursor.execute("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE';")
tables = cursor.fetchall()

print(f"Tablas en la base de datos {database_name}:")
for table in tables:
    print(table[0])

**Insertar documento a la base de datos**

In [ ]:
def insert_document(CategoryID, FilePath, Uploaded, ExtractedText):
    cursor = connection.cursor()
    cursor.execute("""
        INSERT INTO documents (CategoryID, FilePath, Uploaded, ExtractedText)
        VALUES (?, ?, ?, ?)
    """, (CategoryID, FilePath, Uploaded, ExtractedText))
    connection.commit()
    print(f"Se ha insertado el documento {FilePath} en la base de datos")

insert_document(1, r"C:\Users\oswal\Desktop\EpicGames\IP Partner Licensing Agreement.pdf", '2025-03-22', pdf_text)

In [ ]:
def process_document(FilePath, Uploaded):
    extracted_text = exctact_text_from_pdf(FilePath)
    category = classify_document(extracted_text)
    category_id = get_category_id(category)
    insert_document(category_id, FilePath, Uploaded, extracted_text)
    print(f"Se ha procesado el documento {FilePath} y ha sido guardado en la base de datos.")

In [ ]:
def get_category_id_by_name(category_name):
    cursor = connection.cursor()
    cursor.execute("SELECT CategoryID FROM categories WHERE CategoryName = ?", (category_name,))
    result = cursor.fetchone()[0]
    return result[0] if result else None

#
**Uso de GPU/CPU**

In [ ]:
device = 0 if torch.cuda.is_available() else -1
print(f"Using device: {'GPU' if device == 0 else 'CPU'}")

#
**Modelo de Zero Shot Image Classification - Facebook**

**Imagen de acuerdo de pago**

In [ ]:
image_2_show_2 = mpimg.imread(r"C:\Users\oswal\Pictures\Screenshots\Imagen_de_acuerdo_legal.png")
plt.imshow(image_2_show_2)
plt.axis('off')
plt.show()

#
**Ejemplos de funcionalidad**

In [ ]:
def clasificar_imagen_zero_shot(ruta_imagen, etiquetas=None):
    if etiquetas is None:
        etiquetas = [
            "Contrato",
            "Factura",
            "Recibo",
            "Identificación personal",
            "Informe",
            "Currículum / Hoja de vida",
            "Certificado",
            "Licencia o permiso",
            "Correspondencia (cartas, emails)",
            "Documentación legal",
            "Documentación técnica",
            "Manual o guía",
            "Proyecto",
            "Planificación / Agenda",
            "Otros"
        ]

    classifier = pipeline("zero-shot-image-classification", model="facebook/metaclip-b16-fullcc2.5b")
    imagen = Image.open(ruta_imagen)
    resultado = classifier(imagen, candidate_labels=etiquetas)
    return resultado

In [ ]:
ruta = r"C:\Users\oswal\Pictures\Screenshots\Imagen_de_acuerdo_legal.png"
resultado = clasificar_imagen_zero_shot(ruta)
print(resultado)

In [ ]:
ruta = r"C:\Users\oswal\Pictures\Screenshots\acta.png"
resultado = clasificar_imagen_zero_shot(ruta)
print(resultado)

In [ ]:
ruta = r"C:\Users\oswal\Pictures\Screenshots\asamblea.png"
resultado = clasificar_imagen_zero_shot(ruta)
print(resultado)

In [ ]:
ruta = r"C:\Users\oswal\Pictures\Screenshots\certificado.png"
resultado = clasificar_imagen_zero_shot(ruta)
print(resultado)

In [ ]:
ruta = r"C:\Users\oswal\Pictures\Screenshots\onapi.png"
resultado = clasificar_imagen_zero_shot(ruta)
print(resultado)

**Función de clasificar documentos**

In [ ]:
def classify_document(text, categories):
    """
    Clasifica el texto de un documento en una categoría usando clasificación zero-shot.
    
    Args:
        text (str): Texto del documento a clasificar.
        categories (list): Lista de categorías posibles.
    
    Returns:
        str: Categoría predicha para el documento.
    """

    result = classifier(text, candidate_labels=categories)
    return result["labels"][0]


In [ ]:
def get_categories_from_db():
    """"
    Obtiene las categorías almacenadas en la base de datos.

    Returns:
        list: Lista de categorías almacenadas en la base de datos.
    """

    cursor = connection.cursor()
    cursor.execute("SELECT CategoryName FROM dbo.Categories")
    categories = [row[0] for row in cursor.fetchall()]
    return categories

In [ ]:
def classify_and_update_document(DocumentID, ExtractedText):

    categories = get_categories_from_db()
    predicted_category = classify_document(ExtractedText, categories)

    cursor = connection.cursor()
    cursor.execute("""
        UPDATE dbo.Documents
        SET CategoryID = (SELECT CategoryID FROM dbo.Categories WHERE CategoryName = ?)
        WHERE DocumentID = ?
    """, (predicted_category, DocumentID))
    connection.commit()
    print(f"Se ha clasificado el documento {DocumentID} como {predicted_category}")

In [ ]:
DocumentID = 1
ExtractedText = extract_text_from_pdf(r"C:\Users\oswal\Epic Games - Island Creator Program.pdf")
classify_and_update_document(DocumentID, ExtractedText)

In [ ]:
import os
from glob import glob
from openai import OpenAI

# 1. Configurar cliente DeepSeek (OpenAI compatible)
client = OpenAI(
    api_key="sk-1c61d586aa734348952523cf4516d3ac",  # tu API key
    base_url="https://api.deepseek.com"
)

# 2. Función para listar PDFs de una carpeta
def obtener_pdfs_carpeta(carpeta):
    patron = os.path.join(carpeta, "*.pdf")
    return glob(patron)  # devuelve rutas completas de los PDFs [web:54][web:58]

# 3. Función que llama a DeepSeek para analizar los nombres/rutas de los PDFs
def buscar_documentos_inteligente(consulta, carpeta_docs):
    pdf_paths = obtener_pdfs_carpeta(carpeta_docs)

    if not pdf_paths:
        return "No se encontraron PDFs en la carpeta."

    # Construimos un prompt con la lista de archivos
    lista_archivos = "\n".join(f"- {os.path.basename(p)}" for p in pdf_paths)

    prompt = f"""
Tienes esta lista de documentos PDF guardados en la carpeta:

{lista_archivos}

Tarea del usuario: {consulta}

Responde indicando qué archivos parecen más relevantes y por qué.
"""

    respuesta = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": "Eres un asistente que ayuda a elegir documentos relevantes."},
            {"role": "user", "content": prompt},
        ],
        temperature=0.3,
    )

    return respuesta.choices[0].message.content

# 4. Función para mostrar resultados en pantalla
def mostrar_resultados(texto):
    print("=== RESULTADOS ===")
    print(texto)

# 5. Ejemplo de uso
carpeta_docs = "/content/docs"  # ajusta a la carpeta donde pusiste los PDFs
resultados = buscar_documentos_inteligente("Busca documentos PDF del 2025", carpeta_docs)
mostrar_resultados(resultados)
